In [1]:
import os
import sys
from collections import OrderedDict

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib qt
sys.path.append('../muse')
import utils
import scipy

import mne
from mne.io import Raw
from mne.preprocessing import ICA
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
import numpy as np
import pandas as pd
from scipy.integrate import simps
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
band_power = pd.DataFrame(columns = ['Channel', 'Band', 'Absolute_power','relative_power',
                                    'welch\'s','subject','week','pre_filtered_length','filtered_length','question'])

comps = [0,1,1,0,0,0,0,0,0]
session = 1
thresh = 60
sfreq = 256

questions = pd.read_csv('../sub-instructor/behavioral_question-slide_map.csv.csv')

band_power_lst = []

# reading in subject folders
data_subs = [i for i in sorted(os.listdir('..' + os.sep)) if 'sub-' in i and "instructor" not in i]
print(data_subs)
#data_subs = [data_subs[2]] #only to run one at a time
for data_sub in data_subs:
    print(data_sub)
    data_subject = data_sub
    short_sub = data_sub.split('-')[-1]
    data_sesses =  [i for i in sorted(os.listdir('..' + os.sep + data_sub)) if 'ses-' in i]
    print(data_sesses)
    for data_ses in data_sesses:
        print(data_ses)
        try:
            print(data_ses)
            short_ses = data_ses.split('-')[-1]
            print(short_ses)
            data = []
            data_dir = '..' + os.sep + data_sub + os.sep + data_ses
            print(data_dir)
            data_files =  [i for i in sorted(os.listdir(data_dir)) if '_cleaned' in i] #filter for just _cleaned files
            for data_file in data_files:
                print(data_file)
                data.append((short_sub, short_ses, data_file))
            
            marker_lst = [1,2,3,4,5,6,7,8,9,10]
            behavior_df = utils.compileBehavior(questions, data_ses, short_ses, marker_lst, remove = True)   
            
            # make to an argument in compileBehavior, pass in a list of marker to keep, if no, then keep
            # done
            
            data_fnames = [data_dir + os.sep + file for file in data_files]
            avg_dict = dict(AF=[1,2], TP=[0,3])
            
            filt_raw = utils.readRawAndProcess(data_fnames, behavior_df, filt = [0.1, 50.0, thresh], 
                                               sfreq = sfreq, plot = False)
            
            [events, this] = mne.events_from_annotations(filt_raw, regexp=None)
            durations = np.array(filt_raw.annotations.duration[:] * sfreq + events[:,0])
            
            events = utils.eventsPreprocess(events, durations, filt_raw, small_dur = 8, remove = True)
            # make this into eventsPreprocess with defalut being True, 
            # done
            
            raw_list = utils.annotateRaw(events, filt_raw, sfreq, True)
            
            # make this as a return of annotateRaw
            # done
            
            for idx, filt_raw in enumerate(raw_list):

                try:
                    # rename filtEpoch, bad events that make sense in terms of getting bad sections
                    # done
                    filt_raw, pre_filt_length, question_number = utils.filtEpochDropBad(filt_raw, thresh, avg_dict, True)
                    filtered_events = len(filt_raw.events)
                    
                    data = filt_raw.to_data_frame()
                    filtered_length = len(data[['AF']])
                    data = data[['AF','TP']] * 1e-6
                    col_list = ['AF','TP']
                    
                    print(data)
                    
                    params = (band_power, data_subject,data_ses,pre_filt_length,filtered_length, question_number)
                    band_power = utils.power(data, sfreq, col_list, params, range_val = 2) # return the band_power # done
                    
                    head(band_power)
                except Exception as error:
                    print(error)
            
        except Exception as error:
            print(error)
            

['sub-102']
sub-102
['ses-01']
ses-01
ses-01
01
..\sub-102\ses-01
sub-102_EEG_recording_2023-05-25-08.33.38_cleaned.csv
          timestamps     TP9     AF7     AF8    TP10  Right AUX  Marker0_x  \
0       1.685004e+09   0.000   0.000   0.000   0.000        0.0          0   
1       1.685004e+09   0.000   0.000  -0.488  -0.488        0.0          0   
2       1.685004e+09  -1.953  -1.465  -1.465  -1.465        0.0          0   
3       1.685004e+09   2.930   2.441   1.953   1.953        0.0          0   
4       1.685004e+09 -15.625 -14.160 -13.184 -20.996        0.0          0   
...              ...     ...     ...     ...     ...        ...        ...   
612494  1.685054e+09 -35.645 -29.785 -35.156 -55.176        0.0          0   
612495  1.685054e+09 -41.016 -35.645 -30.273 -50.781        0.0          0   
612496  1.685054e+09 -26.367 -16.602 -18.066 -37.109        0.0          0   
612497  1.685054e+09  -9.277  -4.395 -21.973 -31.738        0.0          0   
612498  1.685054e+09 -

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

Fitting ICA took 1.2s.
Applying ICA to Raw instance
    Transforming to ICA space (4 components)
    Zeroing out 1 ICA component
    Projecting back using 4 PCA components
Used Annotations descriptions: ['BAD boundary', 'EDGE boundary', 'bad_acq_skip']
Using matplotlib as 2D backend.
Creating RawArray with float64 data, n_channels=3, n_times=112212
    Range : 123175 ... 235386 =    481.152 ...   919.477 secs
Ready.
Not setting metadata
1168 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1075 events and 129 original time points ...
              AF        TP
0      -0.000006  0.000007
1      -0.000009  0.000010
2      -0.000008  0.000010
3      -0.000005  0.000006
4      -0.000002  0.000002
...          ...       ...
138670  0.000021 -0.000005
138671  0.000014 -0.000004
138672  0.000008 -0.000005
138673  0.000005 -0.000005
138674  0.000008 -0.000006

[138675 rows x 2 columns]
AF - Delta
Absolute Delta power: 0.000000 

    Range : 538106 ... 569833 =   2101.977 ...  2225.910 secs
Ready.
Not setting metadata
330 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 257 events and 129 original time points ...
             AF            TP
0     -0.000009 -1.516223e-06
1     -0.000015  4.099211e-07
2     -0.000018  2.510608e-06
3     -0.000018  2.637449e-06
4     -0.000021  2.064663e-06
...         ...           ...
33148  0.000016 -1.568264e-06
33149  0.000012 -4.423999e-07
33150  0.000007 -3.781774e-07
33151  0.000006 -1.428243e-06
33152  0.000007 -1.892090e-06

[33153 rows x 2 columns]
AF - Delta
Absolute Delta power: 0.000000 uV^2
Relative Delta power: 0.293110
AF - Theta
Absolute Theta power: 0.000000 uV^2
Relative Theta power: 0.036182
AF - Alpha
Absolute Alpha power: 0.000000 uV^2
Relative Alpha power: 0.018103
AF - Beta
Absolute Beta power: 0.000000 uV^2
Relative Beta power: 0.102030
AF - Gamma
Absolute Gamma power: 0.000000 uV^2
Rela

In [3]:
band_power

,Channel,Band,Absolute_power,relative_power,welch's,subject,week,pre_filtered_length,filtered_length,question
0,AF,Delta,9.953473e-12,0.226099,9.953473e-12,sub-102,ses-01,112212,138675,1.0
1,AF,Theta,4.415360e-12,0.100297,4.415360e-12,sub-102,ses-01,112212,138675,1.0
2,AF,Alpha,2.027576e-12,0.046058,2.027576e-12,sub-102,ses-01,112212,138675,1.0
3,AF,Beta,1.285808e-11,0.292078,1.285808e-11,sub-102,ses-01,112212,138675,1.0
4,AF,Gamma,9.750640e-12,0.221491,9.750640e-12,sub-102,ses-01,112212,138675,1.0
5,TP,Delta,1.496685e-12,0.223961,1.496685e-12,sub-102,ses-01,112212,138675,1.0
6,TP,Theta,1.071111e-12,0.160279,1.071111e-12,sub-102,ses-01,112212,138675,1.0
7,TP,Alpha,5.674541e-13,0.084913,5.674541e-13,sub-102,ses-01,112212,138675,1.0
8,TP,Beta,1.618599e-12,0.242204,1.618599e-12,sub-102,ses-01,112212,138675,1.0
9,TP,Gamma,1.182627e-12,0.176966,1.182627e-12,sub-102,ses-01,112212,138675,1.0


Channels marked as bad:
none
